# ఆటోజెన్ టూల్ ఉపయోగం ఉదాహరణ


## అవసరమైన ప్యాకేజీలను దిగుమతి చేసుకోండి


In [ ]:
import os
import json

import requests
from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_core.tools import FunctionTool
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from typing import Any, Callable, Set, Dict, List, Optional

## క్లయింట్‌ను సృష్టించడం

ఈ ఉదాహరణలో, LLM యాక్సెస్ కోసం [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) ఉపయోగిస్తాము.

`model` ను `gpt-4o-mini` గా నిర్వచించారు. GitHub Models మార్కెట్‌ప్లేస్‌లో అందుబాటులో ఉన్న మరో మోడల్‌కు మోడల్‌ను మార్చి వివిధ ఫలితాలను చూడండి.

త్వరిత పరీక్షగా, మేము ఒక సింపుల్ ప్రాంప్ట్‌ను నడుపుతాము - `What is the capital of France`.


In [ ]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## ఫంక్షన్లను నిర్వచించడం

ఈ ఉదాహరణలో, ఏజెంట్‌కు అందుబాటులో ఉన్న సెలవు గమ్యస్థానాల జాబితా మరియు వాటి లభ్యతతో కూడిన ఒక ఫంక్షన్ అనే సాధనానికి ప్రాప్యతను ఇస్తాము.

ఇది ఒక ప్రయాణ ఏజెంట్‌కు, ఉదాహరణకు, ఒక ట్రావెల్ డేటాబేస్‌కు ప్రాప్యత ఉండే పరిస్థితి అని మీరు ఊహించవచ్చు.

ఈ నమూనాను అనుసరించినప్పుడు, ఏజెంట్ కాల్ చేయగలిగే కొత్త ఫంక్షన్లు మరియు సాధనాలను నిర్వచించడానికి ప్రయత్నించండి.


In [ ]:
from typing import Dict, List, Optional


def vacation_destinations(city: str) -> tuple[str, str]:
    """
    Checks if a specific vacation destination is available
    
    Args:
        city (str): Name of the city to check
        
    Returns:
        tuple: Contains city name and availability status ('Available' or 'Unavailable')
    """
    destinations = {
        "Barcelona": "Available",
        "Tokyo": "Unavailable",
        "Cape Town": "Available",
        "Vancouver": "Available",
        "Dubai": "Unavailable",
    }

    if city in destinations:
        return city, destinations[city]
    else:
        return city, "City not found"

# Example usage:
# city, status = vacation_destinations("Barcelona")
# print(f"How about visiting {city}? It's currently {status} there!")

## ఫంక్షన్ టూల్ నిర్వచించడం  
`vacation_destinations` ను `FunctionTool` గా ఉపయోగించడానికి, దాన్ని ఒకటిగా నిర్వచించాలి.  

మేము టూల్ యొక్క వివరణను కూడా అందిస్తాము, ఇది యూజర్ అభ్యర్థించిన పనికి సంబంధించి ఆ టూల్ ఏమి ఉపయోగించబడుతుందో ఏజెంట్ గుర్తించడానికి సహాయపడుతుంది.  


In [ ]:
get_vacations = FunctionTool(
    vacation_destinations, description="Search for vacation destinations and if they are available or not."
)

## ఏజెంట్‌ను నిర్వచించడం

ఇప్పుడు క్రింది కోడ్‌లో ఏజెంట్‌ను సృష్టించవచ్చు. వినియోగదారులు సెలవు గమ్యస్థానాలను కనుగొనడంలో సహాయపడే విధంగా ఏజెంట్‌కు సూచనలు ఇవ్వడానికి `system_message` ను నిర్వచిస్తాము.

మేము `reflect_on_tool_use` పరామితిని true గా సెట్ చేస్తాము. ఇది టూల్ కాల్ యొక్క ప్రతిస్పందనను తీసుకుని, సహజ భాషలో ప్రతిస్పందనను పంపడానికి LLM ను ఉపయోగించడానికి అనుమతిస్తుంది.

పరామితిని false గా సెట్ చేసి తేడా చూడవచ్చు.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[get_vacations],
    system_message="You are a travel agent that helps users find vacation destinations.",
    reflect_on_tool_use=True,
)

## ఏజెంట్‌ను నడపడం

ఇప్పుడు టోక్యోకు ప్రయాణం చేయాలని అడుగుతూ ప్రారంభ వినియోగదారు సందేశంతో ఏజెంట్‌ను నడపవచ్చు.

మీరు ఈ నగర గమ్యస్థానాన్ని మార్చి, నగర లభ్యతకు ఏజెంట్ ఎలా స్పందిస్తుందో చూడవచ్చు.


In [ ]:
async def assistant_run() -> None:
    response = await agent.on_messages(
        [TextMessage(content="I would like to take a trip to Tokyo", source="user")],
        cancellation_token=CancellationToken(),
    )
    print(response.inner_messages)
    print(response.chat_message)


# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**విమర్శ**:  
ఈ పత్రాన్ని AI అనువాద సేవ [Co-op Translator](https://github.com/Azure/co-op-translator) ఉపయోగించి అనువదించారు. మేము ఖచ్చితత్వానికి ప్రయత్నిస్తున్నప్పటికీ, ఆటోమేటెడ్ అనువాదాలలో తప్పులు లేదా అసమానతలు ఉండవచ్చు. దయచేసి, మూల భాషలోని అసలు పత్రాన్ని అధికారం కలిగిన మూలంగా పరిగణించండి. ముఖ్యమైన సమాచారం కోసం, ప్రొఫెషనల్ మానవ అనువాదాన్ని సిఫారసు చేస్తాము. ఈ అనువాదాన్ని ఉపయోగించడం వల్ల కలిగే ఏవైనా అపార్థాలు లేదా తప్పుదారులు కోసం మేము బాధ్యత వహించము.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
